<a href="https://colab.research.google.com/github/Hemalatha-mahesh/Data-Driven-Stock-Analysis-Organizing-Cleaning-and-Visualizing-Market-Trends/blob/main/data_analytics_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pandas

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("/content/full_data_stack.csv")
display(data)

,Ticker,close,date,high,low,month,open,volume
0,ADANIENT,3065.10,2024-04-23 05:30:00,3091.00,3043.10,2024-04,3059.00,1654272
1,ADANIENT,3026.95,2024-04-19 05:30:00,3057.05,2962.50,2024-04,3000.00,1727317
2,ADANIENT,3252.10,2024-04-01 05:30:00,3291.80,3207.85,2024-04,3230.20,1292646
3,ADANIENT,3115.25,2024-04-25 05:30:00,3120.90,3025.65,2024-04,3050.00,1515529
4,ADANIENT,3043.55,2024-04-24 05:30:00,3087.85,3040.00,2024-04,3071.95,605917
...,...,...,...,...,...,...,...,...
14195,WIPRO,519.15,2024-09-04 05:30:00,528.25,516.75,2024-09,528.00,10201064
14196,WIPRO,539.55,2024-09-24 05:30:00,541.90,529.35,2024-09,532.20,6177847
14197,WIPRO,532.45,2024-09-02 05:30:00,542.00,531.85,2024-09,538.40,7867991
14198,WIPRO,534.90,2024-09-23 05:30:00,540.40,532.00,2024-09,539.10,4272145


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14200 entries, 0 to 14199
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  14200 non-null  object 
 1   close   14200 non-null  float64
 2   date    14200 non-null  object 
 3   high    14200 non-null  float64
 4   low     14200 non-null  float64
 5   month   14200 non-null  object 
 6   open    14200 non-null  float64
 7   volume  14200 non-null  int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 887.6+ KB


In [ ]:
#to check missing values#
data.isna().sum()

,0
Ticker,0
close,0
date,0
high,0
low,0
month,0
open,0
volume,0


In [ ]:
! pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 50.1 MB/s eta 0:00:00


In [ ]:
#data insertion sql # titb #
import pandas as pd
import mysql.connector

# === Step 1: Read CSV file ===
csv_file = "/content/full_data_stack.csv"   # Change this to your stock CSV file path
df = pd.read_csv(csv_file)

# Handle missing values (replace NaN with 0 or suitable defaults)
df = df.fillna(0)

# === Step 2: Connect to TiDB Cloud ===
mydb = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3ZnYqfQTEsJNQce.root",
    password="gybKhZEBwPo9Zwxr",
    database="testdb"
)
print("Connected:", mydb.is_connected())

mycursor = mydb.cursor(buffered=True)

# === Step 3: Create table if not exists ===
mycursor.execute("""
CREATE TABLE IF NOT EXISTS stocks (
    Ticker VARCHAR(50),
    date DATETIME,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume BIGINT,
    month VARCHAR(20)
)
""")

# === Step 4: Prepare bulk insert ===
sql = """INSERT INTO stocks (Ticker, date, open, high, low, close, volume, month)
         VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""

# Ensure correct column names from your CSV
values = df[['Ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'month']].values.tolist()

# === Step 5: Execute bulk insert ===
mycursor.executemany(sql, values)
mydb.commit()

print(f"Inserted {mycursor.rowcount} rows into TiDB Cloud")

# === Step 6: Close connection ===
mycursor.close()
mydb.close()


Connected: True
Inserted 14200 rows into TiDB Cloud


In [ ]:
#drop table from titb
import mysql.connector

# === Connect to TiDB Cloud ===
mydb = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3ZnYqfQTEsJNQce.root",
    password="gybKhZEBwPo9Zwxr",
    database="testdb"
)
print("Connected:", mydb.is_connected())

mycursor = mydb.cursor(buffered=True)

# === Drop table ===
mycursor.execute("DROP TABLE IF EXISTS stocks")

print("Table 'stocks' deleted from TiDB Cloud")

# === Close connection ===
mycursor.close()
mydb.close()


Connected: True
Table 'stocks' deleted from TiDB Cloud
